# Move all the Pangeo Legacy stores into a separate BQ table

```{INFO}
The catalog metadata needs some fixing!
```

In [1]:
from leap_data_management_utils.cmip_testing import test_all
from leap_data_management_utils import CMIPBQInterface, IIDEntry
import zarr
import pandas as pd
from tqdm.auto import tqdm
import dask.bag as db
from dask.diagnostics import ProgressBar
from dask.distributed import LocalCluster, Client
from leap_data_management_utils.cmip_catalog import intake_esm_df_to_bq_df

In [2]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 64.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:49743,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 64.00 GiB
Comm: tcp://127.0.0.1:49755,Total threads: 3
Dashboard: http://127.0.0.1:49760/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:49746,


In [4]:
table_id = 'leap-pangeo.cmip6_pgf_ingestion.pangeo_legacy'
bq = CMIPBQInterface(table_id=table_id)

Creating self.table_id ='leap-pangeo.cmip6_pgf_ingestion.pangeo_legacy'


In [5]:
def fix_legacy_df(df: pd.DataFrame) -> pd.DataFrame:
    """There were several errors in the original dataframe. This should fix it and bring it into official CMIP conventions
    Source: https://docs.google.com/document/d/1h0r8RZr_f3-8egBMMh7aqLwy3snpD6_MrDz1q8n5XUk/edit
    1. `member_id` and `variant_label` are mixed up. 
    2. `dcpp_init` year is not mentioned anywhere in the CMIP docs. I suspect this is `sub_experiment_id`
    3. Some values are saved as floats causing issues with generating proper iids
    """
    df = df.copy()
    # df = df.drop(columns='variant_label')
    df = df.rename(columns={'dcpp_init_year':'sub_experiment_id', 'member_id':'variant_label'})
    # convert sub_experiment column to str and prepend an `s` (see: https://github.com/WCRP-CMIP/CMIP6_CVs/blob/master/CMIP6_sub_experiment_id.json)
    df['sub_experiment_id'] = df['sub_experiment_id'].fillna(1e6).astype(int)
    #
    df['sub_experiment_id'] = 's'+df['sub_experiment_id'].astype('str')
    # remove nan values
    df.loc[df['sub_experiment_id'].isin(['s1000000']), "sub_experiment_id"] = 'none'
    return df

In [6]:
df_legacy_raw = pd.read_csv('https://storage.googleapis.com/cmip6/pangeo-cmip6.csv')
df_legacy_fixed_raw = fix_legacy_df(df_legacy_raw)
df_legacy = intake_esm_df_to_bq_df(df_legacy_fixed_raw)

In [7]:
def test(store_url,iid):
    store = zarr.storage.FSStore(store_url)
    try:
        test_all(store, iid, verbose=False)
        test_passed = True
    except Exception as e: 
        test_passed = False
    return test_passed 

In [11]:
# filter for iids that do not exist
# need to do this in batches
iids = df_legacy['instance_id'].tolist()
iids_in_bq = []
batchsize = 10000 #10k seems to be the limit 
iid_batches = [iids[i:i+batchsize] for i in range(0,len(iids), batchsize)]
for iids_batch in tqdm(iid_batches):
    iids_in_bq_batch = bq.iid_list_exists(iids_batch)
    iids_in_bq.extend(iids_in_bq_batch)

  0%|          | 0/52 [00:00<?, ?it/s]

In [12]:
df_legacy_filtered = df_legacy[~df_legacy.instance_id.isin(iids_in_bq)]

In [13]:
df_input = df_legacy_filtered

## Parallel loading and testing xarray datasets and adding to bq in batches

In [14]:
def batch_test_add(batch_df: pd.DataFrame, bq_client: CMIPBQInterface):
    seq = [(a[1].store,a[1].instance_id) for a in batch_df.iterrows()]
    bag = db.from_sequence(seq)
    with ProgressBar():
        results_dask = bag.map(lambda a: (*a, test(*a))).compute()
    results_dask_iid_entry = [IIDEntry(iid=a[1], store=a[0], retracted=False, tests_passed=a[2]) for a in results_dask]
    bq_client.insert_multiple_iids(results_dask_iid_entry)

In [ ]:
n=10000
df_input_batches = [df_input[i:i+n] for i in range(0,len(df_input),n)]
for df_batch in tqdm(df_input_batches):
    batch_test_add(df_batch, bq)

  0%|          | 0/52 [00:00<?, ?it/s]

2024-01-23 21:43:04,346 - distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
2024-01-23 21:43:06,829 - distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
2024-01-23 21:43:29,458 - distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
2024-01-23 21:43:59,925 - distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
2024-01-23 21:44:08,838 - distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
2024-01-23 21:44:09,681 - distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
2024-01-23 21:44:40,584 - distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
2024-01-23 21:45:03,000 - distributed.utils_perf - WARNING - full garbage collections took

## show all legacy stores that had failing tests

In [16]:
df_read = bq.get_latest()
failed_legacy = df_read[df_read['tests_passed'].isin([False])]
failed_legacy

,instance_id,store,timestamp,retracted,tests_passed
6,CMIP6.CMIP.BCC.BCC-ESM1.1pctCO2.r1i1p1f1.Ofx.d...,gs://cmip6/CMIP6/CMIP/BCC/BCC-ESM1/1pctCO2/r1i...,2024-01-24 05:43:17.059340+00:00,False,False
7,CMIP6.CMIP.HAMMOZ-Consortium.MPI-ESM-1-2-HAM.a...,gs://cmip6/CMIP6/CMIP/HAMMOZ-Consortium/MPI-ES...,2024-01-24 05:43:17.059340+00:00,False,False
15,CMIP6.CMIP.HAMMOZ-Consortium.MPI-ESM-1-2-HAM.h...,gs://cmip6/CMIP6/CMIP/HAMMOZ-Consortium/MPI-ES...,2024-01-24 05:43:17.059340+00:00,False,False
38,CMIP6.DAMIP.MRI.MRI-ESM2-0.hist-nat.r5i1p1f1.f...,gs://cmip6/CMIP6/DAMIP/MRI/MRI-ESM2-0/hist-nat...,2024-01-24 05:43:17.059340+00:00,False,False
70,CMIP6.RFMIP.CNRM-CERFACS.CNRM-CM6-1.piClim-con...,gs://cmip6/CMIP6/RFMIP/CNRM-CERFACS/CNRM-CM6-1...,2024-01-24 05:43:17.059340+00:00,False,False
...,...,...,...,...,...
514698,CMIP6.FAFMIP.MIROC.MIROC6.faf-water.r1i1p1f1.f...,gs://cmip6/CMIP6/FAFMIP/MIROC/MIROC6/faf-water...,2024-01-24 03:43:27.830770+00:00,False,False
514703,CMIP6.LUMIP.NCAR.CESM2.deforest-globe.r1i1p1f1...,gs://cmip6/CMIP6/LUMIP/NCAR/CESM2/deforest-glo...,2024-01-24 03:43:27.830770+00:00,False,False
514740,CMIP6.CMIP.MIROC.MIROC6.historical.r10i1p1f1.f...,gs://cmip6/CMIP6/CMIP/MIROC/MIROC6/historical/...,2024-01-24 03:43:27.830770+00:00,False,False
514744,CMIP6.CMIP.MIROC.MIROC6.historical.r7i1p1f1.Of...,gs://cmip6/CMIP6/CMIP/MIROC/MIROC6/historical/...,2024-01-24 03:43:27.830770+00:00,False,False


In [17]:
df_read

,instance_id,store,timestamp,retracted,tests_passed
0,CMIP6.AerChemMIP.BCC.BCC-ESM1.ssp370.r2i1p1f1....,gs://cmip6/CMIP6/AerChemMIP/BCC/BCC-ESM1/ssp37...,2024-01-24 05:43:17.059340+00:00,False,True
1,CMIP6.AerChemMIP.BCC.BCC-ESM1.ssp370.r2i1p1f1....,gs://cmip6/CMIP6/AerChemMIP/BCC/BCC-ESM1/ssp37...,2024-01-24 05:43:17.059340+00:00,False,True
2,CMIP6.AerChemMIP.BCC.BCC-ESM1.ssp370.r3i1p1f1....,gs://cmip6/CMIP6/AerChemMIP/BCC/BCC-ESM1/ssp37...,2024-01-24 05:43:17.059340+00:00,False,True
3,CMIP6.AerChemMIP.NCAR.CESM2-WACCM.histSST.r1i2...,gs://cmip6/CMIP6/AerChemMIP/NCAR/CESM2-WACCM/h...,2024-01-24 05:43:17.059340+00:00,False,True
4,CMIP6.CFMIP.MRI.MRI-ESM2-0.amip-4xCO2.r1i1p1f1...,gs://cmip6/CMIP6/CFMIP/MRI/MRI-ESM2-0/amip-4xC...,2024-01-24 05:43:17.059340+00:00,False,True
...,...,...,...,...,...
514813,CMIP6.ScenarioMIP.CNRM-CERFACS.CNRM-ESM2-1.ssp...,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,2024-01-24 03:43:27.830770+00:00,False,True
514814,CMIP6.ScenarioMIP.CNRM-CERFACS.CNRM-ESM2-1.ssp...,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,2024-01-24 03:43:27.830770+00:00,False,True
514815,CMIP6.ScenarioMIP.CNRM-CERFACS.CNRM-ESM2-1.ssp...,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,2024-01-24 03:43:27.830770+00:00,False,True
514816,CMIP6.ScenarioMIP.CNRM-CERFACS.CNRM-ESM2-1.ssp...,gs://cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM...,2024-01-24 03:43:27.830770+00:00,False,True
